In [ ]:
import os
import glob
import numpy as np
from PIL import Image

from pano import draw_boundary
from pano_lsd_align import panoEdgeDetection, rotatePanorama


# Parameters 

In [ ]:
path_to_image = 'assert/demo.png'
output_dir = 'assert/output_preprocess/'
q_error = 0.7
refine_iter = 3

# Check input arguments validation

In [ ]:
paths = sorted(glob.glob(path_to_image))
if len(paths) == 0:
    print('no images found')

# Check input arguments validation
for path in paths:
    assert os.path.isfile(path), '%s not found' % path
assert os.path.isdir(output_dir), '%s is not a directory' % args.output_dir


# Process each input

In [ ]:
for i_path in paths:
    print('Processing', i_path, flush=True)

    # Load and cat input images
    img_ori = np.array(Image.open(i_path).resize((1024, 512), Image.BICUBIC))[..., :3]

    # VP detection and line segment extraction
    _, vp, _, _, panoEdge, _, _ = panoEdgeDetection(img_ori,
                                                    qError=q_error,
                                                    refineIter=refine_iter)
    panoEdge = (panoEdge > 0)
    
    # Align images with VP
    i_img = rotatePanorama(img_ori / 255.0, vp[2::-1])
    l_img = rotatePanorama(panoEdge.astype(np.float32), vp[2::-1])

    # Dump results
    basename = os.path.splitext(os.path.basename(i_path))[0]
    path_VP = os.path.join(output_dir, '%s_VP.txt' % basename)
    path_i_img = os.path.join(output_dir, '%s_aligned_rgb.png' % basename)
    path_l_img = os.path.join(output_dir, '%s_aligned_line.png' % basename)

    with open(path_VP, 'w') as f:
        for i in range(3):
            f.write('%.6f %.6f %.6f\n' % (vp[i, 0], vp[i, 1], vp[i, 2]))
    Image.fromarray((i_img * 255).astype(np.uint8)).save(path_i_img)
    Image.fromarray((l_img * 255).astype(np.uint8)).save(path_l_img)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
fig = plt.figure(figsize=(20,20))
fig.add_subplot(311)
plt.imshow(img_ori)
fig.add_subplot(312)
plt.imshow(i_img)
fig.add_subplot(313)
plt.imshow(l_img)
plt.show()

In [ ]:
import torch
from model import Encoder, Decoder
from utils_eval import augment, augment_undo
from pano import get_ini_cor, draw_boundary_from_cor_id
from pano_opt import optimize_cor_id

# Model related arguments

In [ ]:
path_prefix = 'ckpt/epoch_30'
device = 'cuda:0'
img_glob = 'assert/output_preprocess/demo_aligned_rgb.png'
line_glob = 'assert/output_preprocess/demo_aligned_line.png'
output_dir = 'assert/output'


#  Data augmented arguments (to improve output quality)

In [ ]:
flip = False
rotate = []
d1 = 21
d2 = 2
post_optimization = False

In [ ]:
device = torch.device(device)

# Check input arguments validation

In [ ]:
for path in glob.glob(img_glob):
    assert os.path.isfile(path), '%s not found' % path
for path in glob.glob(line_glob):
    assert os.path.isfile(path), '%s not found' % path
assert os.path.isdir(output_dir), '%s is not a directory' % output_dir
for rotate in rotate:
    assert 0 <= rotate and rotate <= 1, 'elements in --rotate should in [0, 1]'


# Prepare model

In [ ]:
encoder = Encoder().to(device)
edg_decoder = Decoder(skip_num=2, out_planes=3).to(device)
cor_decoder = Decoder(skip_num=3, out_planes=1).to(device)
encoder.load_state_dict(torch.load('%s_encoder.pth' % path_prefix))
edg_decoder.load_state_dict(torch.load('%s_edg_decoder.pth' % path_prefix))
cor_decoder.load_state_dict(torch.load('%s_cor_decoder.pth' % path_prefix))

#  Load path to visualization

In [ ]:
img_paths = sorted(glob.glob(img_glob))
line_paths = sorted(glob.glob(line_glob))
assert len(img_paths) == len(line_paths), '# of input mismatch for each channels'

# Process

In [ ]:
for i_path, l_path in zip(img_paths, line_paths):
    print('img  path:', i_path)
    print('line path:', l_path)

    # Load and cat input images
    i_img = np.array(Image.open(i_path), np.float32) / 255
    l_img = np.array(Image.open(l_path), np.float32) / 255
    x_img = np.concatenate([
        i_img.transpose([2, 0, 1]),
        l_img.transpose([2, 0, 1])], axis=0)

    # Augment data
    x_imgs_augmented, aug_type = augment(x_img, flip, rotate)

    # Feedforward and extract output images
    with torch.no_grad():
        x = torch.FloatTensor(x_imgs_augmented).to(device)
        en_list = encoder(x)
        edg_de_list = edg_decoder(en_list[::-1])
        cor_de_list = cor_decoder(en_list[-1:] + edg_de_list[:-1])

        edg_tensor = torch.sigmoid(edg_de_list[-1])
        cor_tensor = torch.sigmoid(cor_de_list[-1])

        # Recover the effect from augmentation
        edg_img = augment_undo(edg_tensor.cpu().numpy(), aug_type)
        cor_img = augment_undo(cor_tensor.cpu().numpy(), aug_type)

    # Merge all results from augmentation
    edgmap = edg_img.transpose([0, 2, 3, 1]).mean(0).copy()
    cormap = cor_img.transpose([0, 2, 3, 1]).mean(0)[..., 0].copy()

    # Post processing to extract layout
    cor_id = get_ini_cor(cormap, d1, d2)
    if post_optimization:
        cor_id = optimize_cor_id(cor_id, edgmap, cormap,
                                 num_iters=100, verbose=False)

    # Draw extracted layout on source image
    bon_img = draw_boundary_from_cor_id(cor_id.copy(), i_img * 255)

    # Composite all result in one image
    all_in_one = 0.3 * edgmap + 0.3 * cormap[..., None] + 0.4 * i_img
    all_in_one = draw_boundary_from_cor_id(cor_id.copy(), all_in_one * 255)

    # Dump results
    basename = os.path.splitext(os.path.basename(i_path))[0]
    path_edg = os.path.join(output_dir, '%s_edg.png' % basename)
    path_cor = os.path.join(output_dir, '%s_cor.png' % basename)
    path_bon = os.path.join(output_dir, '%s_bon.png' % basename)
    path_all_in_one = os.path.join(output_dir, '%s_all.png' % basename)
    path_cor_id = os.path.join(output_dir, '%s_cor_id.txt' % basename)

    Image.fromarray((edgmap * 255).astype(np.uint8)).save(path_edg)
    Image.fromarray((cormap * 255).astype(np.uint8)).save(path_cor)
    Image.fromarray(bon_img).save(path_bon)
    Image.fromarray(all_in_one).save(path_all_in_one)
    with open(path_cor_id, 'w') as f:
        for x, y in cor_id:
            f.write('%.6f %.6f\n' % (x, y))

In [ ]:
fig = plt.figure(figsize=(20,20))
fig.add_subplot(411)
plt.imshow(edgmap)
fig.add_subplot(412)
plt.imshow(cormap)
fig.add_subplot(413)
plt.imshow(bon_img)
fig.add_subplot(414)
plt.imshow(all_in_one)
plt.show()

In [ ]:
import numpy as np
from scipy.ndimage import map_coordinates

import open3d
from PIL import Image
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

import functools
from multiprocessing import Pool

from utils_eval import np_coor2xy, np_coory2v
from open3d import JVisualizer


In [ ]:
def xyz_2_coorxy(xs, ys, zs, H, W):
    us = np.arctan2(xs, -ys)
    vs = -np.arctan(zs / np.sqrt(xs**2 + ys**2))
    coorx = (us / (2 * np.pi) + 0.5) * W
    coory = (vs / np.pi + 0.5) * H
    return coorx, coory


def pt_in_poly(poly, pt):
    return poly.contains(Point(pt))


def warp_walls(xy, floor_z, ceil_z, H, W, ppm, alpha):
    all_rgba = []
    all_xyz = []
    for i in range(len(xy)):
        next_i = (i + 1) % len(xy)
        xy_a = xy[i]
        xy_b = xy[next_i]
        xy_w = np.sqrt(((xy_a - xy_b)**2).sum())
        t_h = int(round((ceil_z - floor_z) * ppm))
        t_w = int(round(xy_w * ppm))
        xs = np.linspace(xy_a[0], xy_b[0], t_w)[None].repeat(t_h, 0)
        ys = np.linspace(xy_a[1], xy_b[1], t_w)[None].repeat(t_h, 0)
        zs = np.linspace(floor_z, ceil_z, t_h)[:, None].repeat(t_w, 1)
        coorx, coory = xyz_2_coorxy(xs, ys, zs, H, W)

        plane_texture = np.stack([
            map_coordinates(equirect_texture[..., 0], [coory, coorx], order=1, mode='wrap'),
            map_coordinates(equirect_texture[..., 1], [coory, coorx], order=1, mode='wrap'),
            map_coordinates(equirect_texture[..., 2], [coory, coorx], order=1, mode='wrap'),
            np.zeros([t_h, t_w]) + alpha,
        ], -1)
        plane_xyz = np.stack([xs, ys, zs], axis=-1)

        all_rgba.extend(plane_texture.reshape(-1, 4))
        all_xyz.extend(plane_xyz.reshape(-1, 3))

    return all_rgba, all_xyz


def warp_floor_ceiling(xy, z_floor, z_ceiling, H, W, ppm, alpha, n_thread):
    min_x = xy[:, 0].min()
    max_x = xy[:, 0].max()
    min_y = xy[:, 1].min()
    max_y = xy[:, 1].max()
    t_h = int(round((max_y - min_y) * ppm))
    t_w = int(round((max_x - min_x) * ppm))
    xs = np.linspace(min_x, max_x, t_w)[None].repeat(t_h, 0)
    ys = np.linspace(min_y, max_y, t_h)[:, None].repeat(t_w, 1)
    zs_floor = np.zeros_like(xs) + z_floor
    zs_ceil = np.zeros_like(xs) + z_ceiling
    coorx_floor, coory_floor = xyz_2_coorxy(xs, ys, zs_floor, H, W)
    coorx_ceil, coory_ceil = xyz_2_coorxy(xs, ys, zs_ceil, H, W)

    floor_texture = np.stack([
        map_coordinates(equirect_texture[..., 0], [coory_floor, coorx_floor], order=1, mode='wrap'),
        map_coordinates(equirect_texture[..., 1], [coory_floor, coorx_floor], order=1, mode='wrap'),
        map_coordinates(equirect_texture[..., 2], [coory_floor, coorx_floor], order=1, mode='wrap'),
        np.zeros([t_h, t_w]) + alpha,
    ], -1).reshape(-1, 4)
    floor_xyz = np.stack([xs, ys, zs_floor], axis=-1).reshape(-1, 3)

    ceil_texture = np.stack([
        map_coordinates(equirect_texture[..., 0], [coory_ceil, coorx_ceil], order=1, mode='wrap'),
        map_coordinates(equirect_texture[..., 1], [coory_ceil, coorx_ceil], order=1, mode='wrap'),
        map_coordinates(equirect_texture[..., 2], [coory_ceil, coorx_ceil], order=1, mode='wrap'),
        np.zeros([t_h, t_w]) + alpha,
    ], -1).reshape(-1, 4)
    ceil_xyz = np.stack([xs, ys, zs_ceil], axis=-1).reshape(-1, 3)

    xy_poly = Polygon(xy)
    with Pool(n_thread) as p:
        sel = p.map(functools.partial(pt_in_poly, xy_poly), floor_xyz[:, :2])

    return floor_texture[sel], floor_xyz[sel], ceil_texture[sel], ceil_xyz[sel]

In [ ]:
img = 'assert/output_preprocess/demo_aligned_rgb.png'
layout = 'assert/output/demo_aligned_rgb_cor_id.txt'
camera_height = 1.6
ppm = 120
point_size = 0.0025
alpha = 1.0
threads = 10
ignore_floor = False
ignore_ceiling= False 

In [ ]:
# Reading source (texture img, cor_id txt)
equirect_texture = np.array(Image.open(img)) / 255.0
with open(layout) as f:
    cor_id = np.array([line.split() for line in f], np.float32)

# Convert cor_id to 3d xyz
N = len(cor_id) // 2
H, W = equirect_texture.shape[:2]
floor_z = -camera_height
floor_xy = np_coor2xy(cor_id[1::2], floor_z, W, H)
c = np.sqrt((floor_xy**2).sum(1))
v = np_coory2v(cor_id[0::2, 1], H)
ceil_z = (c * np.tan(v)).mean()

# Warp each wall
all_rgba, all_xyz = warp_walls(floor_xy, floor_z, ceil_z, H, W, ppm, alpha)

# Warp floor and ceiling
if not ignore_floor or not ignore_ceiling:
    fi, fp, ci, cp = warp_floor_ceiling(floor_xy, floor_z, ceil_z, H, W,
                                        ppm=ppm,
                                        alpha=alpha,
                                        n_thread=threads)

    if not ignore_floor:
        all_rgba.extend(fi)
        all_xyz.extend(fp)

    if not ignore_ceiling:
        all_rgba.extend(ci)
        all_xyz.extend(cp)


In [ ]:
print('# of points:', len(all_rgba))

In [ ]:
all_xyz = np.array(all_xyz)
all_rgb = np.array(all_rgba)[:, :3]
pcd = open3d.PointCloud()
pcd.points = open3d.Vector3dVector(all_xyz)
pcd.colors = open3d.Vector3dVector(all_rgb)

In [ ]:
from open3d import JVisualizer
import open3d as o3

In [ ]:
pcd = open3d.geometry.PointCloud()
pcd.points = open3d.utility.Vector3dVector(all_xyz)
pcd.colors = open3d.utility.Vector3dVector(all_rgb)
visualizer = JVisualizer()

In [ ]:
o3.PointCloud = o3.geometry.PointCloud

In [ ]:
visualizer.add_geometry(pcd)
visualizer.show()